In [2]:
import pandas as pd 
from langchain_groq.chat_models import ChatGroq

In [3]:
from dotenv import load_dotenv
import os

# Load the environment variables from the .env file
load_dotenv()

# Access the API key
Groq_token = os.getenv('GROQ_API_KEY')

groq_models = {"llama3-70b": "llama3-70b-8192", "mixtral": "mixtral-8x7b-32768", "gemma-7b": "gemma-7b-it","llama3.1-70b":"llama-3.1-70b-versatile","llama3-8b":"llama3-8b-8192","llama3.1-8b":"llama-3.1-8b-instant","gemma-9b":"gemma2-9b-it"}


In [4]:
import pandas as pd

# Reading the data from the text file into a pandas DataFrame
X_train = pd.read_csv(
    'data_scripts/human+activity+recognition+using+smartphones/UCI HAR Dataset/UCI HAR Dataset/train/X_train.txt',
    sep='\s+', # white space as delimiter
    header=None  # No header row in the file
)

# Display the first few rows of the DataFrame
print("X shape: ",X_train.shape)

y_train=pd.read_csv(
    'data_scripts/human+activity+recognition+using+smartphones/UCI HAR Dataset/UCI HAR Dataset/train/y_train.txt',
    header=None
)

print("y shape: ",y_train.shape)


X shape:  (7352, 561)
y shape:  (7352, 1)


In [5]:
X_test = pd.read_csv(
    'data_scripts/human+activity+recognition+using+smartphones/UCI HAR Dataset/UCI HAR Dataset/test/X_test.txt',
    sep='\s+', # white space as delimiter
    header=None  # No header row in the file
)

y_test=pd.read_csv(
    'data_scripts/human+activity+recognition+using+smartphones/UCI HAR Dataset/UCI HAR Dataset/test/y_test.txt',
    header=None
)

X_test

,0,1,2,3,4,5,6,7,8,9,...,551,552,553,554,555,556,557,558,559,560
0,0.257178,-0.023285,-0.014654,-0.938404,-0.920091,-0.667683,-0.952501,-0.925249,-0.674302,-0.894088,...,0.071645,-0.330370,-0.705974,0.006462,0.162920,-0.825886,0.271151,-0.720009,0.276801,-0.057978
1,0.286027,-0.013163,-0.119083,-0.975415,-0.967458,-0.944958,-0.986799,-0.968401,-0.945823,-0.894088,...,-0.401189,-0.121845,-0.594944,-0.083495,0.017500,-0.434375,0.920593,-0.698091,0.281343,-0.083898
2,0.275485,-0.026050,-0.118152,-0.993819,-0.969926,-0.962748,-0.994403,-0.970735,-0.963483,-0.939260,...,0.062891,-0.190422,-0.640736,-0.034956,0.202302,0.064103,0.145068,-0.702771,0.280083,-0.079346
3,0.270298,-0.032614,-0.117520,-0.994743,-0.973268,-0.967091,-0.995274,-0.974471,-0.968897,-0.938610,...,0.116695,-0.344418,-0.736124,-0.017067,0.154438,0.340134,0.296407,-0.698954,0.284114,-0.077108
4,0.274833,-0.027848,-0.129527,-0.993852,-0.967445,-0.978295,-0.994111,-0.965953,-0.977346,-0.938610,...,-0.121711,-0.534685,-0.846595,-0.002223,-0.040046,0.736715,-0.118545,-0.692245,0.290722,-0.073857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2942,0.310155,-0.053391,-0.099109,-0.287866,-0.140589,-0.215088,-0.356083,-0.148775,-0.232057,0.185361,...,0.074472,-0.376278,-0.750809,-0.337422,0.346295,0.884904,-0.698885,-0.651732,0.274627,0.184784
2943,0.363385,-0.039214,-0.105915,-0.305388,0.028148,-0.196373,-0.373540,-0.030036,-0.270237,0.185361,...,0.101859,-0.320418,-0.700274,-0.736701,-0.372889,-0.657421,0.322549,-0.655181,0.273578,0.182412
2944,0.349966,0.030077,-0.115788,-0.329638,-0.042143,-0.250181,-0.388017,-0.133257,-0.347029,0.007471,...,-0.066249,-0.118854,-0.467179,-0.181560,0.088574,0.696663,0.363139,-0.655357,0.274479,0.181184
2945,0.237594,0.018467,-0.096499,-0.323114,-0.229775,-0.207574,-0.392380,-0.279610,-0.289477,0.007471,...,-0.046467,-0.205445,-0.617737,0.444558,-0.819188,0.929294,-0.008398,-0.659719,0.264782,0.187563


In [12]:
min_value = X_test.min().min()
max_value = X_test.max().max()
print(min_value)
print(max_value)

-1.0
1.0


## Zero Shot

In [14]:
indexes = [0, 5, 32, 38, 58, 65]

for index in indexes:
    # Convert the row to a string representation
    data_str = X_train.iloc[index].to_string()
    query = f"Classify the following accelerometer data into the correct human activity: {data_str}. Print predicted label in as follows: Label: <predicted_label>"
    
    # Initialize and use the language model
    model_name = "llama3-70b"  # Model name
    llm = ChatGroq(model=groq_models[model_name], api_key=Groq_token, temperature=0)
    answer = llm.invoke(query)
    
    # Print the results
    print(f"Index: {index}")
    print(f"LLM Classification: {answer.content.strip()}")
    print(f"Actual Label: {y_train.iloc[index].values[0]}")
    print("\n" + "="*50 + "\n")

Index: 0
LLM Classification: A classification task!

To classify the accelerometer data into a human activity, I'll use a machine learning approach. Since I don't have any additional information about the data, I'll assume it's a multiclass classification problem.

After analyzing the data, I noticed that it's a time series signal with 501 data points. I'll use a simple approach to extract features from the data and then train a classifier to predict the activity label.

Here are the steps I took:

1. **Feature extraction**: I extracted the following features from the time series signal:
	* Mean
	* Standard deviation
	* Variance
	* Skewness
	* Kurtosis
	* Peak-to-peak amplitude
	* Root mean square (RMS) value
2. **Feature selection**: I selected the top 5 features with the highest variance to reduce dimensionality and improve model performance.
3. **Classification**: I trained a random forest classifier on the selected features to predict the activity label.

After training the model, 

## Few Shot

In [7]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import re

# Define the extract_label function
def extract_label(llm_response):
    """Extract the predicted label from the LLM response."""
    match = re.search(r'(?i)label: (\d+)', llm_response)  # The '(?i)' makes the search case-insensitive
    if match:
        return int(match.group(1))
    return None

# Sample indexes from your training set
indexes = [0, 5, 32, 38, 58, 65] 

# Function to get a random subset of row data
def get_random_subset(row, subset_size=10):
    """Returns a random subset of 'subset_size' elements from a row."""
    return row.sample(n=subset_size, random_state=1)

# Prepare few-shot training examples using a small subset of each row
few_shot_examples = []
for index in indexes:
    # Select a random subset of the row
    subset_data = get_random_subset(X_train.iloc[index])
    data_str = subset_data.to_string()
    actual_label = y_train.iloc[index].values[0]
    few_shot_examples.append(f"Data: {data_str}\nLabel: {actual_label}")

# Concatenate the few-shot examples for the prompt
few_shot_prompt = "\n\n".join(few_shot_examples)

clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

# Lists to store results
llm_predictions = []
decision_tree_predictions = []
actual_labels = []

# Test the model on new data
test_indexes = [10, 15, 53, 70, 100, 205, 207,302,310,440  ] # (5, 5,6,6,1,4,4,3,2,1)

for index in test_indexes:
    # Select a random subset of the test row
    subset_data = get_random_subset(X_test.iloc[index])
    data_str = subset_data.to_string()
    query = (f"{few_shot_prompt}\n\n"
             f"Classify the following accelerometer data into the correct human activity.\n\n"
             f"Data: {data_str}\n\n"
             "Please provide the classification in the following format:\n"
             "Label: <predicted_label>")
    
    # Initialize and use the language model
    model_name = "llama3-70b"  # Model name
    llm = ChatGroq(model=groq_models[model_name], api_key=Groq_token, temperature=0)
    answer = llm.invoke(query)
    
    # Extract the LLM prediction
    llm_label = extract_label(answer.content.strip())
    
    # Store LLM prediction and actual label
    llm_predictions.append(llm_label)
    actual_labels.append(y_test.iloc[index].values[0])
    
    # Decision Tree prediction
    decision_tree_prediction = clf.predict([X_test.iloc[index]])[0]
    decision_tree_predictions.append(decision_tree_prediction)
    
    print(f"Index: {index}")
    print(f"LLM Classification: {answer.content.strip()}")
    print(f"Actual Label: {y_test.iloc[index].values[0]}")
    print("\n" + "="*50 + "\n")

Index: 10
LLM Classification: Based on the provided data, I'll try to classify the new accelerometer data into the correct human activity.

After analyzing the patterns and similarities between the given data and the new data, I'm going to make a prediction.

Label: 5
Actual Label: 5


Index: 15
LLM Classification: Based on the provided data, I'll try to classify the new accelerometer data into the correct human activity.

After analyzing the patterns and trends in the given data, I noticed that the Label 5 data tends to have more negative values, while Label 4 data has a mix of positive and negative values. Label 6 data, on the other hand, has a more distinct pattern with some positive values and a few negative values.

Now, let's examine the new data:

Data: 47    -0.982064
439   -0.987811
223   -0.230520
524   -1.000000
427   -0.902356
355   -0.991638
237   -0.212020
172    0.947468
85    -0.987697
532   -0.663203

This new data seems to have a mix of negative and positive values, w

In [21]:
# Convert lists to numpy arrays for easier calculation
llm_predictions = np.array(llm_predictions)
decision_tree_predictions = np.array(decision_tree_predictions)
actual_labels = np.array(actual_labels)

# Calculate accuracy
def calculate_accuracy(predictions, actuals):
    """Calculate the accuracy of predictions compared to actual labels."""
    correct_predictions = np.sum(predictions == actuals)
    accuracy = correct_predictions / len(actuals)
    return accuracy

# Calculate accuracies
llm_accuracy = calculate_accuracy(llm_predictions, actual_labels)
decision_tree_accuracy = calculate_accuracy(decision_tree_predictions, actual_labels)

# Print results
print(f"Few-Shot Learning Accuracy: {llm_accuracy:.2f}")
print(f"Decision Tree Accuracy: {decision_tree_accuracy:.2f}")

Few-Shot Learning Accuracy: 0.30
Decision Tree Accuracy: 0.90


In [9]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Function to generate random data
def generate_random_data(n_samples, n_features, original_data):
    # Generate random data
    random_data = np.random.rand(n_samples, n_features)
    
    # Scale the random data to match the range of the original data
    scaler = MinMaxScaler()
    scaler.fit(original_data)
    scaled_random_data = scaler.transform(random_data)
    
    return scaled_random_data

# Generate random data
n_samples = 10  # Number of random samples to generate
n_features = X_test.shape[1]  # Number of features in the original data
random_data = generate_random_data(n_samples, n_features, X_test)

# Convert random data to DataFrame
random_df = pd.DataFrame(random_data, columns=X_test.columns)

# Lists to store results
random_llm_predictions = []

for index in range(n_samples):
    # Select a random subset of the random row
    subset_data = get_random_subset(random_df.iloc[index])
    data_str = subset_data.to_string()
    query = (f"{few_shot_prompt}\n\n"
             f"Classify the following accelerometer data into the correct human activity.\n\n"
             f"Data: {data_str}\n\n"
             "Please provide the classification in the following format:\n"
             "Label: <predicted_label>")
    
    # Initialize and use the language model
    model_name = "llama3-70b"  # Model name
    llm = ChatGroq(model=groq_models[model_name], api_key=Groq_token, temperature=0)
    answer = llm.invoke(query)
    
    # Extract the LLM prediction
    llm_label = extract_label(answer.content.strip())
    
    # Store LLM prediction
    random_llm_predictions.append(llm_label)
    
    print(f"Random Sample {index + 1}")
    print(f"LLM Classification: {answer.content.strip()}")
    print("\n" + "="*50 + "\n")

# Calculate the distribution of predictions
unique, counts = np.unique(random_llm_predictions, return_counts=True)
distribution = dict(zip(unique, counts))

print("Distribution of predictions for random data:")
for label, count in distribution.items():
    print(f"Label {label}: {count} ({count/n_samples*100:.2f}%)")

Random Sample 1
LLM Classification: A classification task!

After analyzing the provided data, I'll try to classify the new accelerometer data into the correct human activity.

Here's the classification:

Label: 4

My reasoning is based on the patterns and distributions of the values in the provided data. The new data seems to have a similar pattern to the data labeled as 4, with values mostly in the positive range and a few values around 0.7-0.8. This is consistent with the data labeled as 4, which also has a mix of positive values and some values around 0.7-0.8.

Please note that this is a simple classification based on visual inspection and pattern recognition. A more robust approach would involve training a machine learning model on the provided data and using it to classify the new data.


Random Sample 2
LLM Classification: A classification task!

After analyzing the provided data, I'll try to classify the new accelerometer data into the correct human activity.

Here's the classi


1. Demonstration and comparison of Zero-Shot Learning and Few-Shot Learning:

Zero-Shot Learning demonstrated poor performance, often misclassifying activities or providing generic descriptions without specific labels. It struggled to map the raw accelerometer data to human activities without prior examples.

Few-Shot Learning showed improved performance compared to Zero-Shot Learning. It was able to provide specific numeric labels for activities, leveraging the few examples provided. However, its accuracy was still relatively low at 30%.

Few-Shot Learning performs better because it has some examples to learn from, allowing it to establish basic patterns between the data and activity labels. Zero-Shot Learning lacks this advantage, relying solely on general knowledge which isn't sufficient for this specialized task.

2. Quantitative comparison of Few-Shot Learning with Decision Trees:

Few-Shot Learning accuracy: 0.30
Decision Tree accuracy: 0.90

The Decision Tree significantly outperforms Few-Shot Learning, with a 90% accuracy compared to 30%. This is likely because the Decision Tree can effectively learn complex patterns from the full training dataset, while Few-Shot Learning is limited to a small number of examples.

3. Limitations of Zero-Shot and Few-Shot Learning for this task:

- Limited ability to interpret raw sensor data without extensive training
- Difficulty in identifying complex patterns in high-dimensional data
- Reliance on a small number of examples, which may not capture the full variability of activities
- Potential for overfitting to the few examples provided
- Inability to leverage the full training dataset effectively

4. Model classification for entirely new activities:

The model would likely misclassify new activities, attempting to fit them into one of the known categories. It lacks the ability to identify truly novel activities without being explicitly trained to do so.

5. Testing with random data:

If given random data within the same dimensions and range, the model would likely produce unreliable classifications, potentially assigning labels based on superficial similarities to the few examples it has seen, rather than meaningful patterns.


---